In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
import psycopg2
import numpy as np

pd.set_option('display.max_columns',85)

# Import data from Database

In [2]:
conn = psycopg2.connect("dbname=ipo-database user=postgres password=0nlineSQL")

In [3]:
master_df = pd.read_sql_query("SELECT * from master_data;", conn)
master_df.head()

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,deferred_revenue,current_debt,shortterm_debt,total_noncurrent_liabilities,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate
0,BSN.U,2020-09-11,Broadstone Acquisition,Citigroup,$10.00,$9.84,$9.60,-0.04,-$0.16,-$0.40,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEAP.U,2020-09-11,Ribbit LEAP,JPMorgan,$10.00,$11.20,$11.55,0.16,$1.20,$1.55,1.0,Missed,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SNPR.U,2020-09-11,Tortoise Acquisition Corp. II,Barclays/ Goldman Sachs,$10.00,$10.35,$10.51,0.05,$0.35,$0.51,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TWCTU,2020-09-11,TWC Tech Holdings II,Citigroup/ Deutsche Bank Securities,$10.00,$10.08,$10.07,0.01,$0.08,$0.07,1.0,None,Common Stock,TWC Tech Holdings II Corp,NASDAQ,USD,USA,None,None,"Four Embarcadero Center, San Francisco, CA, Un...",2020-12-11,10.52,0.44,0.043651,Gain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,242.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NSH.U,2020-09-10,NavSight Holdings,Credit Suisse,$10.00,$10.00,$10.00,0.00,$0.00,$0.00,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
list(master_df)

['symbol',
 'trade_date',
 'issuer',
 'lead_jointlead_managers',
 'offer_price',
 'opening_price',
 'firstday_close',
 'firstday_percent_pxchng',
 'dollar_change_opening',
 'dollar_change_close',
 'star_ratings',
 'performed',
 'asset_type',
 'company_name',
 'exchange',
 'currency',
 'country',
 'sector',
 'industry',
 'address',
 'three_mth_date',
 'three_mth_ipo',
 'price_change',
 'three_mth_return',
 'price_gain_loss',
 'fiscaldate_end',
 'reported_currency',
 'gross_profit',
 'total_revenue',
 'costof_revenue',
 'costof_goodservices_sold',
 'operating_income',
 'selling_gen_admin',
 'research_dev',
 'operating_expenses',
 'investment_income_net',
 'net_interest_income',
 'interest_income',
 'interest_expense',
 'noninterest_income',
 'othernon_operatingincome',
 'depreciation',
 'dpr_and_amort',
 'income_beforetax',
 'incometax_expense',
 'interest_debt_expense',
 'netincome_cont_operations',
 'comprehensive_income_netoftax',
 'ebit',
 'ebitda',
 'netincome',
 'total_assets',
 't

## Calculated Columns

In [5]:
# Net profit margin
master_df["net_profit_margin"] = master_df["netincome"]/master_df["total_revenue"]
master_df["net_profit_margin"]

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3465   NaN
3466   NaN
3467   NaN
3468   NaN
3469   NaN
Name: net_profit_margin, Length: 3470, dtype: float64

In [6]:
# Calculate Gross Profit Margin = gross_profit/total_revenue
master_df["gross_profit_margin"] = master_df["gross_profit"]/master_df["total_revenue"]
master_df["gross_profit_margin"]

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3465   NaN
3466   NaN
3467   NaN
3468   NaN
3469   NaN
Name: gross_profit_margin, Length: 3470, dtype: float64

### Calculate Debt Asset Ratio:

The formula for the debt to asset ratio is as follows: Debt/Asset = (Short-term Debt + Long-term Debt) / Total Assets. Where: Total Assets may include all current and non-current assets on the company's balance sheet, or may only include certain assets such as Property, Plant & Equipment (PP&E)

In [7]:
# Debt to Asset Ratio
master_df["debt_asset_ratio"] = master_df["short_longterm_debt_total"]/master_df["total_assets"]
master_df

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,total_noncurrent_liabilities,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate,net_profit_margin,gross_profit_margin,debt_asset_ratio
0,BSN.U,2020-09-11,Broadstone Acquisition,Citigroup,$10.00,$9.84,$9.60,-0.04,-$0.16,-$0.40,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LEAP.U,2020-09-11,Ribbit LEAP,JPMorgan,$10.00,$11.20,$11.55,0.16,$1.20,$1.55,1.0,Missed,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SNPR.U,2020-09-11,Tortoise Acquisition Corp. II,Barclays/ Goldman Sachs,$10.00,$10.35,$10.51,0.05,$0.35,$0.51,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TWCTU,2020-09-11,TWC Tech Holdings II,Citigroup/ Deutsche Bank Securities,$10.00,$10.08,$10.07,0.01,$0.08,$0.07,1.0,None,Common Stock,TWC Tech Holdings II Corp,NASDAQ,USD,USA,None,None,"Four Embarcadero Center, San Francisco, CA, Un...",2020-12-11,10.52,0.44,0.043651,Gain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,242.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,NSH.U,2020-09-10,NavSight Holdings,Credit Suisse,$10.00,$10.00,$10.00,0.00,$0.00,$0.00,1.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

### Drop rows with N/As and infinite values

In [8]:
master_df = master_df.dropna(subset=["gross_profit_margin","debt_asset_ratio"])
master_df

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,total_noncurrent_liabilities,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate,net_profit_margin,gross_profit_margin,debt_asset_ratio
15,AUVI,2020-08-31,Applied UV,"Network 1 Financial Securities,",$5.00,$5.75,$11.60,1.32,$0.75,$6.60,0.0,None,Common Stock,"Applied UV, Inc",NASDAQ,USD,USA,Consumer Cyclical,"Furnishings, Fixtures & Appliances","150 N. Macquesten Parkway, Mount Vernon, NY, U...",2020-11-30,5.18,-0.57,-0.099130,Loss,2020-09-30,USD,7.817800e+04,1.560633e+06,1.482455e+06,1.482455e+06,-8.949670e+05,9.251080e+05,48037.0,2.455600e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.055620e+05,5.325850e+05,3.175300e+05,9.929700e+04,0.0,416827.0,8.537970e+05,0.0,5.445513e+06,0.0,1.745910e+05,633.0,6.334308e+06,-4.298970e+05,0.0,0.0,0.0,0.0,9182.0,55814.0,0.0,-45156.0,0.0,-5.581400e+04,4927529.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.573314,0.050094,0.053702
16,HCDI,2020-08-28,Harbor Custom Development,ThinkEquity (a division of Fordham Financial M...,$6.00,$5.50,$7.50,0.25,-$0.50,$1.50,1.0,None,Common Stock,"Harbor Custom Development, Inc",NASDAQ,USD,USA,Real Estate,Real Estate-Development,"11505 Burnham Drive, Gig Harbor, WA, United St...",None,NaN,NaN,NaN,None,2020-09-30,USD,6.226000e+05,7.806500e+06,7.183900e+06,7.183900e+06,-8.356000e+05,0.000000e+00,0.0,8.642100e+06,0.0,-163900.0,0.0,163900.0,0.0,0.0,0.0,...,6.633000e+06,2.858600e+06,3.774400e+06,2.625700e+07,0.0,30031400.0,0.000000e+00,0.0,9.512600e+06,0.0,-2.574500e+06,11957000.0,5.628048e+06,-6.957600e+06,0.0,0.0,0.0,0.0,199400.0,79300.0,0.0,0.0,0.0,1.057000e+05,8871000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.055992,0.079754,0.658664
19,XPEV,2020-08-27,XPeng,Credit Suisse/ J.P. Morgan/ BofA Securities,$15.00,$23.10,$21.22,0.41,$8.10,$6.22,3.0,None,Common Stock,XPeng Inc,NYSE,USD,USA,Consumer Cyclical,Auto Manufacturers,"No. 8 Songgang Road, Guangzhou, China, 510640",2020-11-27,64.28,41.18,1.782684,Gain,2020-09-30,CNY,9.154100e+07,1.990119e+09,1.898578e+09,1.898578e+09,-1.744184e+09,1.203792e+09,635373000.0,3.734303e+09,0.0,19290000.0,23216000.0,3926000.0,0.0,0.0,0.0,...,2.136126e+09,4.092390e+08,1.667490e+09,2.804000e+08,0.0,0.0,1.387595e+09,56844000.0,1.972801e+10,0.0,-1.053500e+10,94000.0,1.426361e+09,7.472520e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.924452e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.577232,0.045998,0.000000
39,BEKE,2020-08-13,KE Holdings,Goldman Sachs/

In [9]:
# Remove infinity values from calculated columns dividing by 0
master_df = master_df.replace([-np.inf, np.inf], np.nan)
master_df

,symbol,trade_date,issuer,lead_jointlead_managers,offer_price,opening_price,firstday_close,firstday_percent_pxchng,dollar_change_opening,dollar_change_close,star_ratings,performed,asset_type,company_name,exchange,currency,country,sector,industry,address,three_mth_date,three_mth_ipo,price_change,three_mth_return,price_gain_loss,fiscaldate_end,reported_currency,gross_profit,total_revenue,costof_revenue,costof_goodservices_sold,operating_income,selling_gen_admin,research_dev,operating_expenses,investment_income_net,net_interest_income,interest_income,interest_expense,noninterest_income,othernon_operatingincome,depreciation,...,total_noncurrent_liabilities,capital_lease_obligations,longterm_debt,current_longterm_debt,longterm_debt_noncurrent,short_longterm_debt_total,other_current_liabilities,other_noncurrent_liabilities,total_shareholder_equity,treasury_stock,retained_earnings,common_stock,common_stockshares_outstanding,operating_cashflow,payments_foroperating_activities,proceeds_fromoperating_activities,changein_operating_liabilities,changein_operating_assets,dpr_depletion_amort,capital_expenditures,changein_receivables,changein_inventory,profit_loss,cashflow_frominvestment,cashflow_fromfinancing,proceeds_fromrepayment_shortterm_debt,payments_forrepurchase_commonstock,payments_forrepurchase_equity,payments_forrepurchase_preferredstock,div_payout,div_payout_commonstock,div_payout_preferredstock,proceeds_fromissuance_commonstock,proceeds_fromissuance_longterm_debtcapital_secnet,proceeds_fromissuance_preferredstock,proceeds_fromrepurchase_equity,proceeds_fromsale_treasurystock,changein_cash_cashequivalents,changein_exchangerate,net_profit_margin,gross_profit_margin,debt_asset_ratio
15,AUVI,2020-08-31,Applied UV,"Network 1 Financial Securities,",$5.00,$5.75,$11.60,1.32,$0.75,$6.60,0.0,None,Common Stock,"Applied UV, Inc",NASDAQ,USD,USA,Consumer Cyclical,"Furnishings, Fixtures & Appliances","150 N. Macquesten Parkway, Mount Vernon, NY, U...",2020-11-30,5.18,-0.57,-0.099130,Loss,2020-09-30,USD,7.817800e+04,1.560633e+06,1.482455e+06,1.482455e+06,-8.949670e+05,9.251080e+05,48037.0,2.455600e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.055620e+05,5.325850e+05,3.175300e+05,9.929700e+04,0.0,416827.0,8.537970e+05,0.0,5.445513e+06,0.0,1.745910e+05,633.0,6.334308e+06,-4.298970e+05,0.0,0.0,0.0,0.0,9182.0,55814.0,0.0,-45156.0,0.0,-5.581400e+04,4927529.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.573314,0.050094,0.053702
16,HCDI,2020-08-28,Harbor Custom Development,ThinkEquity (a division of Fordham Financial M...,$6.00,$5.50,$7.50,0.25,-$0.50,$1.50,1.0,None,Common Stock,"Harbor Custom Development, Inc",NASDAQ,USD,USA,Real Estate,Real Estate-Development,"11505 Burnham Drive, Gig Harbor, WA, United St...",None,NaN,NaN,NaN,None,2020-09-30,USD,6.226000e+05,7.806500e+06,7.183900e+06,7.183900e+06,-8.356000e+05,0.000000e+00,0.0,8.642100e+06,0.0,-163900.0,0.0,163900.0,0.0,0.0,0.0,...,6.633000e+06,2.858600e+06,3.774400e+06,2.625700e+07,0.0,30031400.0,0.000000e+00,0.0,9.512600e+06,0.0,-2.574500e+06,11957000.0,5.628048e+06,-6.957600e+06,0.0,0.0,0.0,0.0,199400.0,79300.0,0.0,0.0,0.0,1.057000e+05,8871000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.055992,0.079754,0.658664
19,XPEV,2020-08-27,XPeng,Credit Suisse/ J.P. Morgan/ BofA Securities,$15.00,$23.10,$21.22,0.41,$8.10,$6.22,3.0,None,Common Stock,XPeng Inc,NYSE,USD,USA,Consumer Cyclical,Auto Manufacturers,"No. 8 Songgang Road, Guangzhou, China, 510640",2020-11-27,64.28,41.18,1.782684,Gain,2020-09-30,CNY,9.154100e+07,1.990119e+09,1.898578e+09,1.898578e+09,-1.744184e+09,1.203792e+09,635373000.0,3.734303e+09,0.0,19290000.0,23216000.0,3926000.0,0.0,0.0,0.0,...,2.136126e+09,4.092390e+08,1.667490e+09,2.804000e+08,0.0,0.0,1.387595e+09,56844000.0,1.972801e+10,0.0,-1.053500e+10,94000.0,1.426361e+09,7.472520e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.924452e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.577232,0.045998,0.000000
39,BEKE,2020-08-13,KE Holdings,Goldman Sachs/

# Preprocess

In [10]:
IPO_df = master_df[["exchange","sector","industry","price_gain_loss","debt_asset_ratio", "gross_profit_margin","net_profit_margin","operating_cashflow"]]
IPO_df

,exchange,sector,industry,price_gain_loss,debt_asset_ratio,gross_profit_margin,net_profit_margin,operating_cashflow
15,NASDAQ,Consumer Cyclical,"Furnishings, Fixtures & Appliances",Loss,0.053702,0.050094,-0.573314,-4.298970e+05
16,NASDAQ,Real Estate,Real Estate-Development,None,0.658664,0.079754,-0.055992,-6.957600e+06
19,NYSE,Consumer Cyclical,Auto Manufacturers,Gain,0.000000,0.045998,-0.577232,7.472520e+08
39,NYSE,Real Estate,Real Estate Services,Gain,0.000000,0.213307,0.138040,9.133566e+09
46,NASDAQ,Technology,Software-Application,None,0.160104,0.315930,-0.037718,1.806600e+07
...,...,...,...,...,...,...,...,...
2680,NASDAQ,Other,Other,Loss,0.000000,0.000000,0.108949,1.441000e+06
2722,NASDAQ,Other,Other,Gain,0.000000,0.000000,0.000000,0.000000e+00
2907,NASDAQ,Other,Other,Gain,0.000000,0.000000,0.168834,1.390000e+05
2932,NASDAQ,Other,Other,Loss,0.000000,0.000000,0.150321,1.613000e+06


In [11]:
IPO_df.shape

(517, 8)

In [12]:
IPO_df = IPO_df.dropna()
IPO_df.shape

(310, 8)

In [13]:
IPO_df.nunique()

exchange                 4
sector                  12
industry                68
price_gain_loss          2
debt_asset_ratio        90
gross_profit_margin    192
net_profit_margin      266
operating_cashflow     256
dtype: int64

# Encoding

In [14]:
# Binary encode
IPO_df_encoded = pd.get_dummies(IPO_df, columns=["exchange","sector","industry","price_gain_loss"])
IPO_df_encoded.head()

,debt_asset_ratio,gross_profit_margin,net_profit_margin,operating_cashflow,exchange_AMEX,exchange_BATS,exchange_NASDAQ,exchange_NYSE,sector_Basic Materials,sector_Communication Services,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Other,sector_Real Estate,sector_Technology,sector_Utilities,industry_Advertising Agencies,industry_Aerospace & Defense,industry_Airlines,industry_Airports & Air Services,industry_Apparel Manufacturing,industry_Asset Management,industry_Auto & Truck Dealerships,industry_Auto Manufacturers,industry_Banks-Regional,industry_Biotechnology,industry_Building Materials,industry_Building Products & Equipment,industry_Capital Markets,industry_Chemicals,industry_Communication Equipment,industry_Computer Hardware,industry_Credit Services,industry_Diagnostics & Research,industry_Discount Stores,industry_Drug Manufacturers-Specialty & Generic,industry_Education & Training Services,industry_Electrical Equipment & Parts,...,industry_Insurance Brokers,industry_Insurance-Diversified,industry_Insurance-Property & Casualty,industry_Integrated Freight & Logistics,industry_Internet Content & Information,industry_Internet Retail,industry_Lodging,industry_Medical Care Facilities,industry_Medical Devices,industry_Metal Fabrication,industry_Mortgage Finance,industry_Oil & Gas Drilling,industry_Oil & Gas Equipment & Services,industry_Oil & Gas Midstream,industry_Other,industry_Packaged Foods,industry_Packaging & Containers,industry_Pharmaceutical Retailers,industry_Pollution & Treatment Controls,industry_REIT-Diversified,industry_Real Estate Services,industry_Real Estate-Development,industry_Rental & Leasing Services,industry_Residential Construction,industry_Resorts & Casinos,industry_Restaurants,industry_Semiconductors,industry_Shell Companies,industry_Software-Application,industry_Software-Infrastructure,industry_Specialty Chemicals,industry_Specialty Industrial Machinery,industry_Specialty Retail,industry_Staffing & Employment Services,industry_Telecom Services,industry_Travel Services,industry_Utilities-Regulated Gas,industry_Utilities-Regulated Water,industry_Utilities-Renewable,industry_Waste Management,price_gain_loss_Gain,price_gain_loss_Loss
15,0.053702,0.050094,-0.573314,-4.298970e+05,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
19,0.000000,0.045998,-0.577232,7.472520e+08,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
39,0.000000,0.213307,0.138040,9.133566e+09,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
51,0.165354,0.090474,-0.124847,-6.337000e+06,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
96,0.000000,0.536819,-0.300338,2.347700e+07,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


# Target Feature

In [15]:
y = IPO_df_encoded.price_gain_loss_Gain
X = IPO_df_encoded.drop(columns=["price_gain_loss_Gain","price_gain_loss_Loss"])

In [16]:
y

15      0
19      1
39      1
51      1
96      1
       ..
2680    0
2722    1
2907    1
2932    0
3020    1
Name: price_gain_loss_Gain, Length: 310, dtype: uint8

In [17]:
X

,debt_asset_ratio,gross_profit_margin,net_profit_margin,operating_cashflow,exchange_AMEX,exchange_BATS,exchange_NASDAQ,exchange_NYSE,sector_Basic Materials,sector_Communication Services,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Other,sector_Real Estate,sector_Technology,sector_Utilities,industry_Advertising Agencies,industry_Aerospace & Defense,industry_Airlines,industry_Airports & Air Services,industry_Apparel Manufacturing,industry_Asset Management,industry_Auto & Truck Dealerships,industry_Auto Manufacturers,industry_Banks-Regional,industry_Biotechnology,industry_Building Materials,industry_Building Products & Equipment,industry_Capital Markets,industry_Chemicals,industry_Communication Equipment,industry_Computer Hardware,industry_Credit Services,industry_Diagnostics & Research,industry_Discount Stores,industry_Drug Manufacturers-Specialty & Generic,industry_Education & Training Services,industry_Electrical Equipment & Parts,...,industry_Health Information Services,industry_Information Technology Services,industry_Insurance Brokers,industry_Insurance-Diversified,industry_Insurance-Property & Casualty,industry_Integrated Freight & Logistics,industry_Internet Content & Information,industry_Internet Retail,industry_Lodging,industry_Medical Care Facilities,industry_Medical Devices,industry_Metal Fabrication,industry_Mortgage Finance,industry_Oil & Gas Drilling,industry_Oil & Gas Equipment & Services,industry_Oil & Gas Midstream,industry_Other,industry_Packaged Foods,industry_Packaging & Containers,industry_Pharmaceutical Retailers,industry_Pollution & Treatment Controls,industry_REIT-Diversified,industry_Real Estate Services,industry_Real Estate-Development,industry_Rental & Leasing Services,industry_Residential Construction,industry_Resorts & Casinos,industry_Restaurants,industry_Semiconductors,industry_Shell Companies,industry_Software-Application,industry_Software-Infrastructure,industry_Specialty Chemicals,industry_Specialty Industrial Machinery,industry_Specialty Retail,industry_Staffing & Employment Services,industry_Telecom Services,industry_Travel Services,industry_Utilities-Regulated Gas,industry_Utilities-Regulated Water,industry_Utilities-Renewable,industry_Waste Management
15,0.053702,0.050094,-0.573314,-4.298970e+05,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,0.000000,0.045998,-0.577232,7.472520e+08,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39,0.000000,0.213307,0.138040,9.133566e+09,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51,0.165354,0.090474,-0.124847,-6.337000e+06,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
96,0.000000,0.536819,-0.300338,2.347700e+07,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,0.000000,0.000000,0.108949,1.441000e+06,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2722,0.000000,0.000000,0.000000,0.000000e+00,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0

In [18]:
X.describe()

,debt_asset_ratio,gross_profit_margin,net_profit_margin,operating_cashflow,exchange_AMEX,exchange_BATS,exchange_NASDAQ,exchange_NYSE,sector_Basic Materials,sector_Communication Services,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Other,sector_Real Estate,sector_Technology,sector_Utilities,industry_Advertising Agencies,industry_Aerospace & Defense,industry_Airlines,industry_Airports & Air Services,industry_Apparel Manufacturing,industry_Asset Management,industry_Auto & Truck Dealerships,industry_Auto Manufacturers,industry_Banks-Regional,industry_Biotechnology,industry_Building Materials,industry_Building Products & Equipment,industry_Capital Markets,industry_Chemicals,industry_Communication Equipment,industry_Computer Hardware,industry_Credit Services,industry_Diagnostics & Research,industry_Discount Stores,industry_Drug Manufacturers-Specialty & Generic,industry_Education & Training Services,industry_Electrical Equipment & Parts,...,industry_Health Information Services,industry_Information Technology Services,industry_Insurance Brokers,industry_Insurance-Diversified,industry_Insurance-Property & Casualty,industry_Integrated Freight & Logistics,industry_Internet Content & Information,industry_Internet Retail,industry_Lodging,industry_Medical Care Facilities,industry_Medical Devices,industry_Metal Fabrication,industry_Mortgage Finance,industry_Oil & Gas Drilling,industry_Oil & Gas Equipment & Services,industry_Oil & Gas Midstream,industry_Other,industry_Packaged Foods,industry_Packaging & Containers,industry_Pharmaceutical Retailers,industry_Pollution & Treatment Controls,industry_REIT-Diversified,industry_Real Estate Services,industry_Real Estate-Development,industry_Rental & Leasing Services,industry_Residential Construction,industry_Resorts & Casinos,industry_Restaurants,industry_Semiconductors,industry_Shell Companies,industry_Software-Application,industry_Software-Infrastructure,industry_Specialty Chemicals,industry_Specialty Industrial Machinery,industry_Specialty Retail,industry_Staffing & Employment Services,industry_Telecom Services,industry_Travel Services,industry_Utilities-Regulated Gas,industry_Utilities-Regulated Water,industry_Utilities-Renewable,industry_Waste Management
count,310.000000,310.000000,310.000000,3.100000e+02,310.000000,310.000000,310.000000,310.000000,310.000000,310.00000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,...,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,0.191143,-0.080342,-20.367528,4.250189e+07,0.003226,0.003226,0.519355,0.474194,0.012903,0.06129,0.077419,0.035484,0.016129,0.070968,0.161290,0.048387,0.406452,0.022581,0.077419,0.009677,0.003226,0.003226,0.003226,0.003226,0.006452,0.009677,0.003226,0.006452,0.003226,0.100000,0.006452,0.003226,0.012903,0.003226,0.003226,0.003226,0.025806,0.012903,0.003226,0.012903,0.022581,0.006452,...,0.009677,0.006452,0.003226,0.006452,0.003226,0.003226,0.045161,0.022581,0.003226,0.003226,0.016129,0.003226,0.003226,0.003226,0.009677,0.003226,0.406452,0.009677,0.006452,0.006452,0.003226,0.006452,0.012903,0.003226,0.003226,0.003226,0.003226,0.006452,0.006452,0.003226,0.035484,0.022581,0.003226,0.003226,0.003226,0.003226,0.006452,0.003226,0.003226,0.003

# Split Training and Testing sets

In [19]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Logistic Regression
our basic logistic regression parameters, which include:

- The solver parameter is set to 'lbfgs', which is an algorithm for learning and optimization. The particular solver isn't very important in this example, but note that a number of optimizers exist.
- The max_iter parameter will be set to 200 iterations, which will give the model sufficient opportunity to converge on effective weights

In [20]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.603


# Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.538


# SVM
- issues running when operating cash flow was added

In [22]:
# from sklearn.svm import SVC

# # Create the SVM model
# svm = SVC(kernel='linear')

# # Train the model
# svm.fit(X_train, y_train)

# # Evaluate the model
# y_pred = svm.predict(X_test_scaled)
# print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

# Deep Learning

In [23]:
len(X_train_scaled[0])

88

In [24]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  number_input_features*2
hidden_nodes_layer2 = number_input_features

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
8/8 [==============================] - 0s 1ms/step - loss: 0.7070 - accuracy: 0.5329
Epoch 2/100
8/8 [==============================] - 0s 2ms/step - loss: 0.6212 - accuracy: 0.6596
Epoch 3/100
8/8 [==============================] - 0s 2ms/step - loss: 0.5488 - accuracy: 0.7256
Epoch 4/100
8/8 [==============================] - 0s 1ms/step - loss: 0.5474 - accuracy: 0.7059
Epoch 5/100
8/8 [==============================] - 0s 2ms/step - loss: 0.5339 - accuracy: 0.7039
Epoch 6/100
8/8 [==============================] - 0s 1ms/step - loss: 0.5037 - accuracy: 0.7151
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: 0.5014 - accuracy: 0.7068
Epoch 8/100
8/8 [==============================] - 0s 1ms/step - loss: 0.4825 - accuracy: 0.7390
Epoch 9/100
8/8 [==============================] - 0s 2ms/step - loss: 0.4798 - accuracy: 0.7469
Epoch 10/100
8/8 [==============================] - 0s 3ms/step - loss: 0.4865 - accuracy: 0.7184
Epoch 11/100
8/8 [===========

8/8 [==============================] - 0s 1ms/step - loss: 0.3784 - accuracy: 0.7848
Epoch 84/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3299 - accuracy: 0.8278
Epoch 85/100
8/8 [==============================] - 0s 997us/step - loss: 0.3329 - accuracy: 0.8305
Epoch 86/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3673 - accuracy: 0.7905
Epoch 87/100
8/8 [==============================] - 0s 2ms/step - loss: 0.3238 - accuracy: 0.8250
Epoch 88/100
8/8 [==============================] - 0s 2ms/step - loss: 0.3601 - accuracy: 0.7912
Epoch 89/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3253 - accuracy: 0.8402
Epoch 90/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3481 - accuracy: 0.8005
Epoch 91/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3170 - accuracy: 0.8467
Epoch 92/100
8/8 [==============================] - 0s 997us/step - loss: 0.3394 - accuracy: 0.8506
Epoch 93/100
8/8 [===========